# Regression Notebook with Census Data

The purpose of this notebook is to find explanatory variables that influence the volume of groundwater testing in California. More information on data and context can be found in the  (1) Background and (2) Data Notion pages to better understand the data being used.

## Specialty Imports
* Zipcode and SearchEngine will be utilized for a reverse geocoding function

In [1]:
#Set Up
import pandas as pd
import numpy as np

# GIS/Location Related"
from uszipcode import Zipcode
from uszipcode import SearchEngine
search = SearchEngine(simple_zipcode=True)

# Regression
import statsmodels.api as sm

/srv/app/venv/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


## Reverse geocoder function
Credits: https://stackoverflow.com/questions/60324730/reverse-geo-coding-without-google-api-for-zip-codes


In [2]:
def coord_to_zip(lat,lng):
    """ 
    Reverse geocoder that takes a lat and longitude value and finds the zipcode
    """
    zip_code = []
    for x,y in zip(lat,lng):
        result = search.by_coordinates(lat=x, lng=y, radius=14, returns=5)
        zip_code.append(result[0].zipcode)
    return zip_code

# Loading Data
We will be using the cleaned data from Ro that has various ACS forms/Census forms compiled by zipcode. Learn more at the Data tab of the Notion page!
https://github.com/Ro-Data/Ro-Census-Summaries-By-Zipcode/blob/master/
 
* _income_ measures wealth and taxes by zipcode
* _demo_ provides demographics by zipcode
* _econ_ measures employment by zipcode


In [3]:
income = pd.read_csv('assorted_data/income_by_zip.csv')
income = income[['Zipcode','TotalWages']].rename({'Zipcode': 'Zip Code'}, axis = 1)

demo = pd.read_csv('census_data/demo.txt', sep = '\t').dropna()#fillna(method='ffill')
demo['Zip Code'] = demo['ZCTA5']

econ = pd.read_csv('census_data/econ.txt', sep = '\t').dropna()#fillna(method='ffill')
econ['Zip Code'] = demo['ZCTA5']

In [4]:
# The following hashed out code will reverse-geocode the GAMA data but
# due to the long run time.

# gama['Zip Code'] = coord_to_zip(gama['latitude'], gama['longitude'])
# gama.to_csv("gama_2019_geocoded.csv")


# Loads pre-geocoded GAMA data
gama = pd.read_csv('gama_data/gama_2019_geocoded.csv').dropna(axis = 0)

# Cleaning and Wrangling

### Aggregating the number of tests and then counting 

In [5]:
gama_test_counts = pd.DataFrame(gama.groupby(['Zip Code'])['results'].agg('count')).reset_index().astype(int)
gama_test_counts.head()

,Zip Code,results
0,91950,82
1,92058,82
2,92061,76
3,92071,74
4,92086,157


### Merging the dataframes, then using more readable labels

In [6]:
gama_demo = pd.merge(left = gama_test_counts, right = demo, on = 'Zip Code')
g_d_econ  = pd.merge(left = gama_demo, right = econ, on = 'Zip Code')

final = g_d_econ.rename({'results': "Number of Tests",}, axis = 1)
final.shape

(18, 380)

# Checkpoint

It is not ideal that we could only find information for 18 zip codes. A naive single-variable OLS will likely suffer from large standard errors. Luckily, the aim of this project is to _explore_ possible features rather than conclusively determine features. In the future, I would want to spend more time cleaning and exploring the data. Ideas I have currently are:

1. Running PCA on the 379 features and then cross validating to measure my error. I haven't investigated all the features and it might be that the census data is inherently low rank. For example, there may be many linear combinations of income measures. 
2. Stepwise Regression, Backward Selection, or Forward Selection for feature selection

However let's see if the data we have, again which is by zipcode, describes a region. For example, what if all the zipcodes were in the South Bay? From that perspective, we actually have as rich of a granularity we could ask for!


Click Here to View in Your Browser
 ==> https://www.easymapmaker.com/map/0191e9e8c3e6855fe1e390003db8c360
 


![final_map](./images/final_map.png "Rough Plot of Final")

Nice! While a more rigorous data cleaning process could probability give us more results for more regions in California. This "EDA" of sorts provides the following insights

* We can filter the 'final' dataframe for Fresno-tangent locations such that we could use our results to explain the Fresno region.
* Future cleaning could allow us to create models and associative claims for regions within California


# A Simple Model for Fresno, California

Out of curiosity let's run a simple single-variable OLS and feature slection process for the sake of exploration. Zipcodes that are in Fresno and the 'final' dataframe begin with '93xxx'


In [30]:
fresno = final[final['Zip Code'] > 93000]

In [31]:
corr = fresno.corr()
corr_target = abs(corr["Number of Tests"])

#Selecting highly correlated features
relevant_features = corr_target[corr_target>0.5].index

# Running the OLS

In [29]:
Y = fresno[["Number of Tests"]]


X_train = fresno[['hispanic_or_latino_and_race_detailed-population-not_hispanic_or_latino_other_total_population']]

X_with_const = sm.add_constant(X_train)

linear_model = sm.OLS(Y, X_with_const).fit()
linear_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        Number of Tests   R-squared:                       0.271
Model:                            OLS   Adj. R-squared:                  0.199
Method:                 Least Squares   F-statistic:                     3.726
Date:                Thu, 21 May 2020   Prob (F-statistic):             0.0824
Time:                        16:23:08   Log-Likelihood:                -61.280
No. Observations:                  12   AIC:                             126.6
Df Residuals:                      10   BIC:                             127.5
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
=================================================================================================================================================================
                                                                                                    coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------------------------------------------------------------
const                                                                                            81.7300     15.093      5.415      0.000      48.101     115.359
hispanic_or_latino_and_race_detailed-population-not_hispanic_or_latino_other_total_population     0.4366      0.226      1.930      0.082      -0.067       0.941
==============================================================================
Omnibus:                        3.353   Durbin-Watson:                   1.763
Prob(Omnibus):                  0.187   Jarque-Bera (JB):                1.492
Skew:                           0.860   Prob(JB):                        0.474
Kurtosis:                       3.165   Cond. No.                         79.7
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

# References
* PCA and Small Sample Sizes https://stats.stackexchange.com/questions/116132/regression-with-very-small-sample-size

* Feature Selection https://towardsdatascience.com/feature-selection-techniques-in-regression-model-26878fe0e24e

